<a href="https://colab.research.google.com/github/GoryachevDaniil/Movies_ReccomendedSystem/blob/main/Movies_ReccomendedSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
from scipy.sparse import csr_matrix

from sklearn.neighbors import NearestNeighbors

In [ ]:
movies = pd.read_csv('/content/drive/MyDrive/Movies_ReccomendedSystem/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Movies_ReccomendedSystem/ratings.csv')

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
movies.drop('genres', axis=1, inplace=True)
ratings.drop('timestamp', axis=1, inplace=True)

In [ ]:
user_item_matrix = ratings.pivot(index='movieId', columns='userId', values='rating')
user_item_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
user_item_matrix.fillna(0, inplace=True)
user_item_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
user_votes = ratings.groupby('userId')['rating'].count()
movie_votes = ratings.groupby('movieId')['rating'].count()

In [ ]:
movie_mask = movie_votes[movie_votes > 10].index
user_mask = user_votes[user_votes > 50].index

In [ ]:
user_item_matrix = user_item_matrix.loc[movie_mask, :]
user_item_matrix = user_item_matrix.loc[:, user_mask]

In [ ]:
user_item_matrix

userId,1,4,6,7,10,11,15,16,17,18,...,600,601,602,603,604,605,606,607,608,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5,3.5,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
176371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
177765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
csr_data = csr_matrix(user_item_matrix.values)

In [ ]:
user_item_matrix = user_item_matrix.rename_axis(None, axis=1).reset_index()
user_item_matrix.head()

,movieId,1,4,6,7,10,11,15,16,17,...,600,601,602,603,604,605,606,607,608,610
0,1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
1,2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
2,3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
4,6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0


In [ ]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [ ]:
recomendations = 10

search_word = 'Jumanji'

In [ ]:
movie_search = movies[movies['title'].str.contains(search_word)]
movie_search

,movieId,title
1,2,Jumanji (1995)
9636,179401,Jumanji: Welcome to the Jungle (2017)


In [ ]:
movie_id = movie_search.iloc[0]['movieId']

movie_id = user_item_matrix[user_item_matrix['movieId'] == movie_id].index[0]
movie_id

1

In [ ]:
distances, indices = knn.kneighbors(csr_data[movie_id], n_neighbors=recomendations + 1)

In [ ]:
distances, indices

(array([[0.        , 0.37701322, 0.41374278, 0.41616445, 0.42056321,
         0.42542803, 0.43500669, 0.44343178, 0.46261167, 0.46965353,
         0.47425265]]),
 array([[  1, 173, 175, 227, 217, 262, 268, 260, 253, 149,  72]]))

In [ ]:
distances_list = distances.squeeze().tolist()
indices_list = indices.squeeze().tolist()

In [ ]:
indices_distances = list(zip(indices_list, distances_list))
print(indices_distances[:5])

[(1, 0.0), (173, 0.3770132181136182), (175, 0.4137427842741309), (227, 0.41616445019173887), (217, 0.4205632052050985)]


In [ ]:
indices_distances_sorted = sorted(indices_distances, key=lambda x : x[1], reverse=True)[:-1]
indices_distances_sorted

[(72, 0.4742526487742069),
 (149, 0.46965352842617836),
 (253, 0.46261166941937715),
 (260, 0.443431779832277),
 (268, 0.43500669079100873),
 (262, 0.4254280334192815),
 (217, 0.4205632052050985),
 (227, 0.41616445019173887),
 (175, 0.4137427842741309),
 (173, 0.3770132181136182)]

In [ ]:
recomended_list = []

for elem in indices_distances_sorted:
  matrix_movie_id = user_item_matrix.iloc[elem[0]]['movieId']
  id = movies[movies['movieId'] == matrix_movie_id].index
  title = movies.iloc[id]['title'].values[0]
  dist = elem[1]
  recomended_list.append({'Title' : title, 'Distance' : dist})

In [ ]:
recomended_list

[{'Title': 'Casper (1995)', 'Distance': 0.4742526487742069},
 {'Title': 'Stargate (1994)', 'Distance': 0.46965352842617836},
 {'Title': 'Nightmare Before Christmas, The (1993)',
  'Distance': 0.46261166941937715},
 {'Title': 'Home Alone (1990)', 'Distance': 0.443431779832277},
 {'Title': 'Beauty and the Beast (1991)', 'Distance': 0.43500669079100873},
 {'Title': 'Aladdin (1992)', 'Distance': 0.4254280334192815},
 {'Title': 'Jurassic Park (1993)', 'Distance': 0.4205632052050985},
 {'Title': 'Mrs. Doubtfire (1993)', 'Distance': 0.41616445019173887},
 {'Title': 'Mask, The (1994)', 'Distance': 0.4137427842741309},
 {'Title': 'Lion King, The (1994)', 'Distance': 0.3770132181136182}]

In [ ]:
df_recomendetion = pd.DataFrame(recomended_list, index=range(1, recomendations + 1))

In [ ]:
df_recomendetion

,Title,Distance
1,Casper (1995),0.474253
2,Stargate (1994),0.469654
3,"Nightmare Before Christmas, The (1993)",0.462612
4,Home Alone (1990),0.443432
5,Beauty and the Beast (1991),0.435007
6,Aladdin (1992),0.425428
7,Jurassic Park (1993),0.420563
8,Mrs. Doubtfire (1993),0.416164
9,"Mask, The (1994)",0.413743
10,"Lion King, The (1994)",0.377013
